In [1]:
# packages
import pandas as pd
import sqlite3
from datetime import timedelta
from dtype_dictionaries import create_dtype_dict
import math

## Metdata (weather) imputation
All the data is weather related and has a time dimension. 

#### Numeric values
(a) Sparse missings

Impute by interpolating the time series from the same Station

(b) Dens missings

Impute by taking averaging nearby stations


#### Factors
(a) Sparse missings

Impute the

In [2]:
conn = sqlite3.connect('../data/formatted_zone/formatted_zone.db')
table_name = 'Metoffice_01_22_2023-10-13'
dtype_dict, date_columns = create_dtype_dict(table_name)
df = pd.read_sql_query(f"SELECT * FROM \"{table_name}\";", conn, parse_dates=date_columns)

dtype_dict, date_columns = create_dtype_dict('weather_station_locations_2023-10-20')
loc_df = pd.read_sql_query(f"SELECT * FROM \"{'weather_station_locations_2023-10-20'}\";", conn, parse_dates=date_columns)
conn.close()

Metoffice_01_22_2023-10-13
weather_station_locations_2023-10-20


In [5]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Assuming the Earth is a flat plane, calculate the distance using Pythagoras' theorem
    lat_diff = lat2 - lat1
    lon_diff = lon2 - lon1
    distance = math.sqrt(lat_diff**2 + lon_diff**2)

    return distance

In [20]:
interpolation_threshold = 0.8
station_counts = df.Station_name.value_counts() / 2

for column in df.columns:
    # count null values per station for this column
    station_null_counts = df.loc[df.loc[:,column].isna()].Station_name.value_counts()
    
    # iterate through the dataframe
    for i, value in enumerate(df[column]):
        if pd.isna(value):
            # handle the null value
            station = df.loc[i, "Station_name"]
            date = df.iloc[i].Date

            # check if this station should be interpolated for this column
            perc_missing = station_null_counts[station] / station_counts[station]
            if  perc_missing < interpolation_threshold: # TODO check if we can interpolate
                # interpolate by the station over days
                
                start_date = date - timedelta(days = 2)
                end_date = date + timedelta(days = 2)
                
                interpolation_df = df.loc[(df.Station_name == station) & 
                          (df.Date.dt.time == date.time()) & 
                          (df.Date >= start_date) &
                          (df.Date <= end_date)]
                
                # TODO change to interpolate the series instead of mean
                interpolated_value = interpolation_df.loc[:,column].mean()
            else: 
                # mean of nearby stations
                lat = loc_df.loc[loc_df.SITE == station].LAT
                lon = loc_df.loc[loc_df.SITE == station].LON
                loc_df['Distance'] = loc_df.apply(lambda row: euclidean_distance(lat, lon, row['LAT'], row['LON']), axis=1)

                # Sort the DataFrame by distance
                loc_df = loc_df.sort_values(by='Distance')

                # Select the top three closest stations
                closest_stations = list(loc_df.head(4).SITE)
                closest_df = df.loc[(df.Date == date) & df.Station_name.apply(lambda x: x in closest_stations)]

                interpolated_value = closest_df.loc[:,column].mean()

In [22]:
interpolation_df

,Date,Station_no,Station_name,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW
1674,2022-01-24 12:00:00,30,Manston,1034.1,E,4.0,8.0,4.0,1.9
1744,2022-01-25 12:00:00,30,Manston,1034.3,NNW,2.0,8.0,4.7,2.3
1814,2022-01-26 12:00:00,30,Manston,NaN,None,NaN,NaN,NaN,NaN
1884,2022-01-27 12:00:00,30,Manston,1026.7,None,NaN,8.0,9.9,7.9
1954,2022-01-28 12:00:00,30,Manston,1035.7,WSW,5.0,6.0,7.5,5.5


In [21]:
closest_df

,Date,Station_no,Station_name,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW
2138,2022-01-31 12:00:00,4,Loch Glascarnoch,1020.9,WNW,21.0,8.0,3.0,-0.8
2139,2022-01-31 12:00:00,5,Aviemore,1021.5,None,NaN,8.0,3.4,-1.2
2141,2022-01-31 12:00:00,7,Tulloch Bridge,1023.1,NW,8.0,8.0,3.5,-0.3
2143,2022-01-31 12:00:00,9,Leuchars,1021.6,W,13.0,4.0,5.7,-2.5
